# Rees - Jones (2017) American Economic Review
## Mistaken Play in the Deferred Acceptance Algorithm: Implications for Positive Assortative Matching

DAは選好順序が真の能力なり性質なりをすべて反映していれば、strategy-proof、すなわちマッチングに参加するプレイヤーは自身の選好を偽らずにそのまま報告することが弱支配戦略となることを保証するアルゴリズムである。

しかし、現実のマッチング問題にDAを用いると、必ずしもすべてのプレイヤーが真の選好順序を報告するわけではないことが多数の事例において報告されている。

例えば、学校と生徒のマッチングを考える。

この時学校側が生徒に対して持つ選好順序はなんらかの統一テストなどの結果であることが予想できる。しかし、一回きりの統一テストの結果が生徒の真の能力を完全に反映させるわけではないことは容易に想像できる。体調不良や問題への適性などにより真の能力順序とはいくらか異なる順序をテストの結果として得られていると想定するのがむしろ自然なことである。

本論文はこのような状況下で、生徒側が真の選好順序を報告しないことにより社会厚生が改善する可能性のあることを指摘している。

具体的には、能力値の低い学生群が自身の選好を偽って報告した場合に、全員が真の選好順序を報告する倍位よりも社会厚生が高くなることをシミュレーションで示している。以下では論文中で行われたシミュレーションを、シミュレーション回数を少なくして再現する。